## Enhanced SciCite Distillation with Cosine Similarity

**Objective**: Fine-tune student model using teacher reasoning with cosine similarity alignment

In [7]:
%pip install -q transformers datasets accelerate peft bitsandbytes sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [1]:
import nbformat
print(nbformat.__version__)

5.10.4


In [2]:
import pandas as pd
from datasets import Dataset
import torch
import time
import os
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    pipeline
)
from dotenv import load_dotenv
load_dotenv()

c:\Users\Shawn Kok\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
def load_dataset_with_context(partition_path: str) -> Dataset:
    df = pd.read_csv(partition_path)
    return Dataset.from_pandas(df[["sectionName", "string", "id", "model_classification", "reasoning"]])

train_dataset = load_dataset_with_context("./merged_dataset.csv")

In [4]:
%run student_eval.ipynb

Device set to use cpu


```json
{"classification": "background", "reasoning": "The text establishes a gap in knowledge regarding the interaction between frataxin and the Fe-S cluster biosynthesis components. It highlights the lack of direct, one-to-one interactions, suggesting this is an area requiring further investigation."}
```
('background', 'The text establishes a gap in knowledge regarding the interaction between frataxin and the Fe-S cluster biosynthesis components. It highlights the lack of direct, one-to-one interactions, suggesting this is an area requiring further investigation.')


In [5]:
hf_token = os.getenv('HUGGINGFACE_API_KEY')
model_id = "google/gemma-3-1b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    token=hf_token
)

: 

## Data Collection
The collate function converts raw data into model-ready tensors

In [ ]:
def collate_fn(examples):
    batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }
    
    metadata = {
        "sectionName": [],
        "string": [],
        "teacher_reasoning": []
    }
    
    for example in examples:
        prompt = f"Classify this citation:\nSection: {example['sectionName']}\nText: {example['string']}\nClassification:"
        tokenized = tokenizer(
            prompt,
            padding="max_length",
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )
        
        batch["input_ids"].append(tokenized["input_ids"])
        batch["attention_mask"].append(tokenized["attention_mask"])
        batch["labels"].append(tokenized["input_ids"].clone())
        
        metadata["sectionName"].append(example["sectionName"])
        metadata["string"].append(example["string"])
        metadata["teacher_reasoning"].append(example["reasoning"])
    
    batch["input_ids"] = torch.cat(batch["input_ids"], dim=0)
    batch["attention_mask"] = torch.cat(batch["attention_mask"], dim=0)
    batch["labels"] = torch.cat(batch["labels"], dim=0)
    batch.update(metadata)
    
    return batch

In [ ]:
call_pipe

In [6]:
class CosineSimilarityDistiller(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.student_pipe = None
        self.step_counter = 0
        
    def setup_pipeline(self):
        if not self.student_pipe:
            self.student_pipe = pipeline(
                "text-generation",
                model=self.model,
                tokenizer=self.tokenizer,
                device=self.model.device
            )
        
    def compute_loss(self, model, inputs, return_outputs=False):
        # Standard language modeling loss
        outputs = model(**inputs)
        loss = outputs.loss
        
        # Cosine similarity alignment every 5 steps
        if self.step_counter % 5 == 0:
            self.setup_pipeline()
            
            # Generate student outputs
            student_reasonings = []
            for section, text in zip(inputs["sectionName"], inputs["string"]):
                _, reasoning = call_pipe(self.student_pipe, section, text)
                student_reasonings.append(reasoning)
            
            # Get embeddings
            teacher_embeds = self.get_embeddings(inputs["teacher_reasoning"])
            student_embeds = self.get_embeddings(student_reasonings)
            
            # Calculate cosine similarity
            cos_sim = torch.nn.functional.cosine_similarity(teacher_embeds, student_embeds)
            similarity_loss = 1 - cos_sim.mean()
            
            # Combine losses
            total_loss = loss + (0.5 * similarity_loss)
            
            if self.step_counter % 10 == 0:
                print(f"Step {self.step_counter} - "
                      f"LM Loss: {loss:.4f} | "
                      f"Similarity Loss: {similarity_loss:.4f} | "
                      f"Total Loss: {total_loss:.4f}")
        else:
            total_loss = loss
        
        self.step_counter += 1
        return (total_loss, outputs) if return_outputs else total_loss
    
    def get_embeddings(self, texts):
        inputs = self.tokenizer(
            texts,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        ).to(self.model.device)
        
        with torch.no_grad():
            outputs = self.model(**inputs, output_hidden_states=True)
            return outputs.hidden_states[-1][:, 0, :]  # CLS token embedding

In [7]:
training_args = TrainingArguments(
    output_dir="cosine-distilled",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    fp16=True,
    logging_steps=10,
    save_strategy="steps",
    eval_steps=200,
    remove_unused_columns=False
)

In [8]:
trainer = CosineSimilarityDistiller(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=collate_fn,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()